In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.4.1 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow-gpu==2.4.1


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
import os

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

actions = os.listdir(DATA_PATH)

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
actions = np.array(actions)

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'itching': 0,
 'cough': 1,
 'nausea': 2,
 'acidity': 3,
 'skin_rash': 4,
 'back_pain': 5,
 'congestion': 6,
 'continuous_sneezing': 7,
 'dizziness': 8,
 'vomitting': 9,
 'runny_nose': 10,
 'diarrhoea': 11,
 'stomach_pain': 12,
 'depression': 13,
 'throat_irritation': 14,
 'inflammatory_nails': 15,
 'sweating': 16,
 'tiredness': 17,
 'chills': 18,
 'indigestion': 19,
 'headache': 20}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
np.array(sequences).shape

(630, 30, 1662)

In [14]:
np.array(labels).shape

(630,)

In [15]:
X = np.array(sequences)

In [16]:
X.shape

(630, 30, 1662)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [20]:
y_test.shape

(32, 21)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [22]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

2022-12-12 15:03:36.074261: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-12-12 15:03:36.074304: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-12-12 15:03:36.074362: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [23]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

2022-12-12 15:03:36.223357: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-12 15:03:36.223611: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 15:03:36.225303: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [24]:
res = [.7, 0.2, 0.1]

In [25]:
actions[np.argmax(res)]

'itching'

In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [27]:
model.fit(X_train, y_train, epochs=300, callbacks=[tb_callback])

2022-12-12 15:03:37.475861: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-12 15:03:37.476493: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz


Epoch 1/300
 1/19 [>.............................] - ETA: 2:35 - loss: 3.0500 - categorical_accuracy: 0.0000e+00

2022-12-12 15:03:46.428889: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-12-12 15:03:46.428930: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.


 2/19 [==>...........................] - ETA: 10s - loss: 3.3737 - categorical_accuracy: 0.0469     

2022-12-12 15:03:46.818567: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-12-12 15:03:47.093945: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-12-12 15:03:47.212665: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: Logs/train/plugins/profile/2022_12_12_15_03_47
2022-12-12 15:03:47.299954: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to Logs/train/plugins/profile/2022_12_12_15_03_47/hackerpro.trace.json.gz
2022-12-12 15:03:47.405138: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: Logs/train/plugins/profile/2022_12_12_15_03_47
2022-12-12 15:03:47.414007: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to Logs/train/plugins/profile/2022_12_12_15_03_47/hackerpro.memory_profile.json.gz
2022-12-12 15:03:47.416743: I tensorf

19/19 [==============================] - 16s 413ms/step - loss: 3.8092 - categorical_accuracy: 0.0643
Epoch 2/300
19/19 [==============================] - 5s 275ms/step - loss: 3.1626 - categorical_accuracy: 0.0652
Epoch 3/300
19/19 [==============================] - 6s 306ms/step - loss: 2.9503 - categorical_accuracy: 0.1192
Epoch 4/300
19/19 [==============================] - 7s 373ms/step - loss: 2.8794 - categorical_accuracy: 0.0787
Epoch 5/300
19/19 [==============================] - 6s 296ms/step - loss: 2.6420 - categorical_accuracy: 0.1398
Epoch 6/300
19/19 [==============================] - 6s 310ms/step - loss: 2.3375 - categorical_accuracy: 0.2530
Epoch 7/300
19/19 [==============================] - 5s 272ms/step - loss: 2.2080 - categorical_accuracy: 0.2489
Epoch 8/300
19/19 [==============================] - 6s 329ms/step - loss: 1.9470 - categorical_accuracy: 0.3447
Epoch 9/300
19/19 [==============================] - 6s 308ms/step - loss: 1.9887 - categorical_accuracy: 0

KeyboardInterrupt: 

In [ ]:
model.summary()

In [28]:
model.save('symptom.h5')